In [1]:
# 0. Settings
!pip install laion-clap
!pip install torch
!pip install torchvision
!pip install torchaudio
import laion_clap
import glob
import json
import torch
import numpy as np
import torchaudio
import tarfile
import zipfile
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
import os
import glob


# Instrument Classification

In [2]:
# 1. labels/audio/text files path  

# files

class_index_dict_path = 'labels/drumsc.json'
zip_files_path = 'data/drum1440.zip'
unzip_files_folder_path = './data/drum1440'
upzip_audio_files_path = 'data/drum1440/**/*.wav'
upzip_text_files_path = 'data/drum1440/**/*.json'
column_name = 'Instrument' # 'genre', 'artist'


In [3]:
# 2. Pretrained Model calling
model = laion_clap.CLAP_Module(enable_fusion=True)
model.load_ckpt()

with zipfile.ZipFile(zip_files_path) as zip:
    zip.extractall(unzip_files_folder_path)
class_index_dict = {k:v for k, v in json.load(open(class_index_dict_path)).items()}
audio_files = sorted(glob.glob(upzip_audio_files_path, recursive=True))
json_files = sorted(glob.glob(upzip_text_files_path, recursive=True)) 
ground_truth_idx = [class_index_dict[json.load(open(jf))[column_name][0]] for jf in json_files]

/home/jeong_soonju/default/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForS

Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

In [4]:
print('='*50)
print(class_index_dict)
print('audio files 갯수:',len(audio_files))
print('json_files 갯수:',len(json_files))
print('ground_truth_idx 갯수:',len(ground_truth_idx))
print('ground_truth_idx :' ,ground_truth_idx)

{'clap': 0, 'hihat': 1, 'snare': 2, 'kick': 3, 'tom': 4, 'cymbal': 5}
audio files 갯수: 1440
json_files 갯수: 1440
ground_truth_idx 갯수: 1440
ground_truth_idx : [5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 5, 5, 5, 5, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 

In [5]:
audio_files

['data/drum1440/cdrip_cymbal_crash_grainy.wav',
 'data/drum1440/cdrip_cymbal_reverse_01.wav',
 'data/drum1440/cdrip_cymbal_reverse_02.wav',
 'data/drum1440/cdrip_cymbal_reverse_03.wav',
 'data/drum1440/cdrip_cymbal_reverse_cat_sneeze.wav',
 'data/drum1440/cdrip_cymbal_ride_909.wav',
 'data/drum1440/cdrip_cymbal_ride_909_glitch.wav',
 'data/drum1440/cdrip_cymbal_splash_01.wav',
 'data/drum1440/cdrip_cymbal_splash_02.wav',
 'data/drum1440/cdrip_kick_01.wav',
 'data/drum1440/cdrip_kick_02.wav',
 'data/drum1440/cdrip_kick_03.wav',
 'data/drum1440/cdrip_kick_04_friendly_ghost.wav',
 'data/drum1440/cdrip_kick_05_trancey.wav',
 'data/drum1440/cdrip_snare_01.wav',
 'data/drum1440/cdrip_snare_02.wav',
 'data/drum1440/cdrip_snare_03.wav',
 'data/drum1440/cdrip_snare_04.wav',
 'data/drum1440/cdrip_snare_05.wav',
 'data/drum1440/cdrip_snare_06.wav',
 'data/drum1440/cdrip_snare_07.wav',
 'data/drum1440/cdrip_snare_08.wav',
 'data/drum1440/convex_kick_one_shot_bad_kids.wav',
 'data/drum1440/convex_k

In [9]:
# Get audio embeddings - pickle
audio_embed_list = []

audio_files_path = os.path.join(unzip_files_folder_path, "**/*.wav")
audio_files = sorted(glob.glob(audio_files_path, recursive=True))
for audio_file in audio_files:
    with open(audio_file, 'rb') as f:
        audio_data = f.read()
    audio_embed = model.get_audio_embedding_from_filelist(x = audio_files[:100], use_tensor=False)
    audio_embed_list.append(audio_embed)
with open('audio_embeddings.pkl', 'wb') as f:
    pickle.dump(audio_embed_list, f)

/home/jeong_soonju/default/lib/python3.9/site-packages/torchaudio/transforms/_transforms.py:611: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 400.00 MiB (GPU 0; 39.43 GiB total capacity; 2.93 GiB already allocated; 166.31 MiB free; 3.00 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF